In [ ]:
        ### TA (Technical Analysis Library) ###

from ta.trend import SMAIndicator, EMAIndicator, MACD, ADXIndicator
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import BollingerBands, AverageTrueRange
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

class TAUtils:
    """Utilidades para calcular y visualizar indicadores técnicos usando TA"""
    
    @staticmethod
    def sma(prices, period=20):
        """Calcula la Media Móvil Simple (SMA)"""
        indicator = SMAIndicator(prices, window=period)
        return indicator.sma_indicator()
    
    @staticmethod
    def ema(prices, period=20):
        """Calcula la Media Móvil Exponencial (EMA)"""
        indicator = EMAIndicator(prices, window=period)
        return indicator.ema_indicator()
    
    @staticmethod
    def macd(prices, fast_period=12, slow_period=26, signal_period=9):
        """Calcula MACD (Moving Average Convergence/Divergence)"""
        indicator = MACD(prices, window_fast=fast_period, window_slow=slow_period, window_sign=signal_period)
        macd = indicator.macd()
        signal = indicator.macd_signal()
        hist = indicator.macd_diff()
        return macd, signal, hist
    
    @staticmethod
    def rsi(prices, period=14):
        """Calcula el índice de fuerza relativa (RSI)"""
        indicator = RSIIndicator(prices, window=period)
        return indicator.rsi()
    
    @staticmethod
    def bollinger_bands(prices, period=20, num_std=2):
        """Calcula las Bandas de Bollinger"""
        indicator = BollingerBands(prices, window=period, window_dev=num_std)
        upper = indicator.bollinger_hband()
        middle = indicator.bollinger_mavg()
        lower = indicator.bollinger_lband()
        return upper, middle, lower
    
    @staticmethod
    def stochastic(high, low, close, k_period=14, d_period=3, smooth_k=3):
        """Calcula el Oscilador Estocástico"""
        indicator = StochasticOscillator(high, low, close, window=k_period, smooth_window=smooth_k)
        k = indicator.stoch()
        d = indicator.stoch_signal()
        return k, d
    
    @staticmethod
    def adx(high, low, close, period=14):
        """Calcula el Índice de Movimiento Direccional Promedio (ADX)"""
        indicator = ADXIndicator(high, low, close, window=period)
        return indicator.adx()
    
    @staticmethod
    def atr(high, low, close, period=14):
        """Calcula el Rango Promedio Verdadero (ATR)"""
        indicator = AverageTrueRange(high, low, close, window=period)
        return indicator.average_true_range()
    
    @staticmethod
    def detect_candlestick_patterns(open_prices, high, low, close):
        """
        TA no tiene detección de patrones de velas incorporada,
        así que implementamos algunos patrones comunes manualmente
        """
        patterns = {}
        
        # Crear DataFrame para facilitar los cálculos
        df = pd.DataFrame({
            'open': open_prices,
            'high': high,
            'low': low,
            'close': close
        })
        
        # Calcular algunas métricas útiles para la detección de patrones
        df['body'] = abs(df['close'] - df['open'])
        df['range'] = df['high'] - df['low']
        df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
        df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
        df['body_rel'] = df['body'] / df['range']
        df['is_bullish'] = df['close'] > df['open']
        
        # Detectar Doji
        patterns['Doji'] = ((df['body'] / df['range']) < 0.1).astype(int)
        
        # Detectar Hammer (versión simplificada)
        patterns['Hammer'] = (
            (df['body_rel'] < 0.3) &  # Cuerpo pequeño
            (df['lower_shadow'] > 2 * df['body']) &  # Sombra inferior larga
            (df['upper_shadow'] < 0.1 * df['body'])  # Sin sombra superior
        ).astype(int)
        
        # Detectar Shooting Star (versión simplificada)
        patterns['Shooting Star'] = (
            (df['body_rel'] < 0.3) &  # Cuerpo pequeño
            (df['upper_shadow'] > 2 * df['body']) &  # Sombra superior larga
            (df['lower_shadow'] < 0.1 * df['body'])  # Sin sombra inferior
        ).astype(int)
        
        # No podemos implementar fácilmente patrones más complejos sin rolling windows
        # Para patrones más avanzados, es mejor usar TA-Lib o Pandas TA
        
        return patterns
    
    @staticmethod
    def plot_sma(prices, periods=[20, 50, 200], figsize=(12, 6)):
        """Grafica el precio y múltiples SMAs"""
        plt.figure(figsize=figsize)
        plt.plot(prices, label='Precio', alpha=0.7)
        
        for period in periods:
            sma = TAUtils.sma(prices, period)
            plt.plot(sma, label=f'SMA {period}')
        
        plt.title('Precio y Medias Móviles Simples (SMA) - TA')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    @staticmethod
    def plot_macd(prices, fast_period=12, slow_period=26, signal_period=9, figsize=(12, 6)):
        """Grafica el MACD"""
        macd, signal, hist = TAUtils.macd(prices, fast_period, slow_period, signal_period)
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, gridspec_kw={'height_ratios': [3, 1]})
        
        # Graficar precios
        ax1.plot(prices)
        ax1.set_title('Precio')
        ax1.grid(True, alpha=0.3)
        
        # Graficar MACD
        ax2.plot(macd, label='MACD')
        ax2.plot(signal, label='Señal')
        ax2.bar(np.arange(len(hist)), hist, label='Histograma', alpha=0.5)
        ax2.axhline(y=0, color='r', linestyle='-', alpha=0.3)
        ax2.set_title('MACD - TA')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()